<center><b>IMPLEMENTING LSTM USING NUMPY</b><center>
<hr>
<ul>
<li>https://agustinus.kristia.de/techblog/2016/08/12/lstm-backprop/</li>
<li>https://www.youtube.com/watch?v=8HyCNIVRbSU&ab_channel=TheA.I.Hacker-MichaelPhi</li>

In [171]:
with open('./shakespeare.txt', 'r') as file:
    data = file.read()

vocab = list(set(data))
ix_to_char = {i:ch for i, ch in enumerate(vocab)}
char_to_ix = {ch:i for i, ch in enumerate(vocab)}

data_size = len(data)
vocab_size = len(vocab)

In [172]:
# hyperparameters
hidden_size = 100 # size of hidden layer of neurons
seq_length = 25 # number of steps to unroll the RNN for
learning_rate = 1e-1

In [173]:
import numpy as np 
W = {}
W['Wf'] = np.random.randn(hidden_size + vocab_size, hidden_size)*0.01
W['Wi'] = np.random.randn(hidden_size + vocab_size, hidden_size)*0.01
W['Wo'] = np.random.randn(hidden_size + vocab_size, hidden_size)*0.01
W['Wc'] = np.random.randn(hidden_size + vocab_size, hidden_size)*0.01
W['Wy'] = np.random.randn(hidden_size, vocab_size)*0.01

W['bf'] = np.zeros((1,hidden_size))
W['bi'] = np.zeros((1,hidden_size))
W['bo'] = np.zeros((1,hidden_size))
W['bc'] = np.zeros((1,hidden_size))
W['by'] = np.zeros((1,vocab_size))



In [174]:
def encode(char, vocab_size):
    one_hot = np.zeros((1,vocab_size))
    one_hot[:,char_to_ix[char]] = 1
    return one_hot

In [175]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

In [176]:
def forward(xs, W, hs, cs):
    X = np.column_stack((hs, xs))
    hf = sigmoid(np.dot(X, W['Wf']) + W['bf'])
    hi = sigmoid(np.dot(X, W['Wi']) + W['bi'])
    ho = sigmoid(np.dot(X, W['Wo']) + W['bo'])
    hc = sigmoid(np.dot(X, W['Wc']) + W['bc'])
    cs = hf*cs + hi*hc
    hs = ho*np.tanh(cs)
    ys = np.dot(hs, W['Wy']) + W['by']
    ps = np.exp(ys)
    ps = ps/np.sum(ps)

    return hf, hi, ho, hc, hs, cs, ys, ps

In [177]:
def one_step(input_char, target_char, W, vocab_size, hs, cs):
    xs = encode(char=input_char, vocab_size=vocab_size)
    hf, hi, ho, hc, hs, cs, ys, ps = forward(xs, W, hs, cs)
    loss = -np.log(ps[:,char_to_ix[target_char]])
    return xs, hf, hi, ho, hc, hs, cs, ys, ps, loss


In [178]:
def step(inputs, targets, W, vocab_size, hs, cs):
    loss = 0 
    xs, hf, hi, ho, hc, hs, ys, ps, cs = {}, {}, {}, {}, {}, {}, {}, {}, {}
    hs[-1] = np.zeros((1,hidden_size))
    cs[-1] = np.zeros((1,hidden_size))

    for t in range(len(inputs)):
        input_char = inputs[t]
        target_char = targets[t]
        xs[t], hf[t], hi[t], ho[t], hc[t], hs[t], cs[t], ys[t], ps[t], step_loss = one_step(input_char=input_char,
                                                                target_char=target_char,
                                                                W=W,
                                                                vocab_size=vocab_size,
                                                                hs=hs[t-1],
                                                                cs=cs[t-1])
        loss+=step_loss

    return xs, hf, hi, ho, hc, hs, cs, ys, ps, loss


In [179]:
def clip_grad(grads):
    for grad in grads.keys():
        grads[grad] = np.clip(grads[grad], -5, 5)
    return grads


In [180]:
def grad(W, xs, hf, hi, ho, hc, hs, cs, ps, targets):
    grads = {}
    for key in W.keys():
        grads[key] = np.zeros_like(W[key])
    
    dhnext = np.zeros_like(hs[0])
    dcnext = np.zeros_like(cs[0])

    for t in reversed(range(len(targets))):
        target_char = targets[t]
        dy = np.copy(ps[t])
        dy[:,char_to_ix[target_char]]-=1

        grads['Wy']+=np.dot(hs[t].T, dy)
        grads['by']+=dy

        dh = np.dot(dy, W['Wy'].T) + dhnext
        
        # Gradient wrt ho
        dho = np.tanh(cs[t])*dh
        dho = sigmoid(ho[t])*(1-sigmoid(ho[t]))*dho

        # Gradient wrt cs
        dc = ho[t]*np.tanh(cs[t])*dh + dcnext

        # Gradient wrt hf
        dhf = sigmoid(hf[t])*(1-sigmoid(hf[t]))*cs[t-1]*dc

        # Gradient wrt hi
        dhi = sigmoid(hi[t])*(1-sigmoid(hi[t]))*hc[t]*dc

        # Gradient wrt ci
        dhc = (1-np.tanh(hc[t])**2)*hi[t]*dc

        X = np.column_stack((hs[t], xs[t]))

        grads['Wf']+=np.dot(X.T, dhf)
        grads['bf']+=dhf

        grads['Wi']+=np.dot(X.T, dhi)
        grads['bi']+=dhi

        grads['Wo']+=np.dot(X.T, dho)
        grads['bo']+=dho

        grads['Wc']+=np.dot(X.T, dhc)
        grads['bc']+=dhc


    grads = clip_grad(grads)

    return grads

In [181]:
def sample(hs, cs, xs, n, W, vocab_size):
    xs = encode(xs, vocab_size)
    outputs = []

    for t in range(n):
        _, _, _, _, hs, cs, _, p = forward(xs, W, hs, cs)
        ix = np.random.choice(range(vocab_size), p=p.ravel())
        outputs.append(ix_to_char[ix])
        xs = np.zeros_like(xs)
        xs[:,ix]=1

    return outputs


In [183]:
n,p = 0,0
mVars = {}
for key in W.keys():
  mVars[key] = np.zeros_like(W[key])

smooth_loss = -np.log(1.0/vocab_size)*seq_length # loss at iteration 0

while True:
  # prepare inputs (we're sweeping from left to right in steps seq_length long)
  if p+seq_length+1 >= len(data) or n == 0: 
    hs = np.zeros((1,hidden_size)) # reset RNN memory
    cs = np.zeros((1,hidden_size))
    p = 0 # go from start of data
  inputs = data[p:p+seq_length]
  targets = data[p+1:p+seq_length+1]

  # sample from the model now and then
  if n % 100 == 0:
    txt = sample(hs, cs, inputs[0], 200, W, vocab_size)
    txt = ''.join(txt)
    print(txt)

  # forward seq_length characters through the net and fetch gradient
  xs, hf, hi, ho, hc, hs, cs, ys, ps, loss = step(inputs=inputs, targets=targets, W=W, 
                              vocab_size=vocab_size, hs=hs, cs=cs)
  grads = grad(xs=xs, hf=hf, hi=hi, ho=ho, hc=hc, hs=hs, cs=cs, ps=ps, W=W, targets=targets)

  hs = hs[len(targets)-1]
  cs = cs[len(targets)-1]
  smooth_loss = smooth_loss * 0.999 + loss * 0.001
  if n % 100 == 0: 
    print(f'iter {n}, loss: {smooth_loss}') # print progress

  # perform parameter update with Adagrad
  for key in W.keys():
    param = W[key]
    dparam = grads[key]
    mem = mVars[key]

    mem += dparam * dparam
    param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update

  p += seq_length # move data pointer
  n += 1 # iteration counter 

 a'ento
BaC' wval
HouCthltelnst ibirwnuprt? faos hwrsr mlrclonkutu rs s 
qltCeaee! yaxneh p;yr.rh s he , erd ghlaiw ookoenv nAoe
bayorce  g  Eonk uenh dx Ma :s 
i apye e yis zn e p ohai?slroowh a vhic
iter 0, loss: [104.32888462]
banin
Irtaze hathhalglle O  ng A?pehe :o
Oiduthn

sh Cene s inta
sgpnisy, uarnY inssaty.
 gha catoulelrh e Oed :o.
Fi? , G
Fain ? weB razafnhe
'iadne, d molumaretvo: 'epphe voon,  od ryo ptc tiplCiton
iter 100, loss: [101.19416718]
ves
: Wilbst namanoy Nond
lerthatl'ewhy: piO'e ,hmomutozelsld e dortsis !r  tue, bains staadly
veYowoe uun
Whaltifo t lgh Ie reitstt ispr thee tolte ilftres  i,t
hiWtilnaklity?
fo
atfo, izevetl' ieve 
iter 200, loss: [98.21376819]
tr'gowaun:
 bacdl oe; caYlels thI sbs Uoe ! y zoben  tiuern;

Fiur uf mube okrssh
wot,
thvelostr SoMeby .
ThtrCid FiFils tablcsAn,: PoUesc
arw tt isre Ci plnlorr , 'rmo'usaury moony utm te!
whitrs:
es
iter 300, loss: [95.48867862]
td yoNuneaf porian mod. do rsr thee e tlohutmu
 rez,n y, opol athye Colile,r

KeyboardInterrupt: 